In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from numpy import mean
from numpy import std
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_validate
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import StratifiedKFold
from pipelinehelper import PipelineHelper
from sklearn.decomposition import PCA

from collections import Counter

from sklearn.metrics import balanced_accuracy_score

In [ ]:
data = pd.read_excel('/Users/andrei-macpro/Documents/Data/classification/accoustics/extraction_3.0/meal_child_prosody.xlsx',index_col=0)
data = data.dropna()
#data = data.drop('label', axis=1)

In [ ]:
data = pd.read_excel('/Users/andrei-macpro/Documents/Data/classification/accoustics/mfcc_child_meal.xlsx',index_col=0)
data = data.dropna()

In [ ]:
combined = pd.concat([data, data2], join='outer',axis=1)
combined = combined.dropna()
#combined = combined.drop('label', axis=1)

In [ ]:
data

In [ ]:
combined['label']

In [ ]:
#X = combined.iloc[:,:-1].to_numpy()
x = StandardScaler().fit_transform(X)
pca = PCA(n_components=5)
principalcomponentst = pca.fit_transform(x)

In [ ]:
print('Explained variation per principal component: {}'.format(pca.explained_variance_ratio_))


In [ ]:
X= principalcomponentst

In [ ]:
X = data.iloc[:,:-1].to_numpy()
y = np.array([0 if x=='no_rad' else 1 for x in data.iloc[:,-1]])
groups = data.index

In [ ]:
len(data.columns)

In [ ]:

##!!!!!!!! this is the pipeline!!!
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm, tree
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
best_estimators = []

#accuracies = []


results = []
predictions = []
#X,y, groups = shuffle(X,y,data.index ,random_state=42)
X_shuffled, y_shuffled, groups_shuffled = shuffle(X,y,groups ,random_state=42)
cv = GroupKFold(n_splits=5)
pipe = Pipeline([
    ('scaler',StandardScaler()), 
    ('classifier', PipelineHelper([
        ('svm', SVC(class_weight='balanced')), 
        ('lr', LogisticRegression(class_weight='balanced')),
        ('tree', tree.DecisionTreeClassifier(class_weight='balanced')),
        ('forest', RandomForestClassifier(class_weight='balanced')),
        ('linear', LinearDiscriminantAnalysis()),
        ('qudratic', QuadraticDiscriminantAnalysis()),
        ('naive', GaussianNB())
    ])),
    ])
params = {
    'classifier__selected_model':pipe.named_steps['classifier'].generate({
          'svm__kernel': ['linear','rbf'],#'svm__C':[0.001, 0.01, 0.1, 1, 10, 100], 'svm__gamma':[0.001, 0.01, 0.1, 1, 10, 100],
     #   'lr__penalty':['l1','l2'], 'lr__solver':['liblinear'],
      #                  'lr__C':np.logspace(-4,4,20)
        
    })
}
search = GridSearchCV(pipe, params, scoring = ['accuracy','balanced_accuracy','f1', 'precision','recall'],refit='balanced_accuracy', cv =cv, n_jobs=-1)

search.fit(X_shuffled, y_shuffled, groups = groups_shuffled)
print(search.best_score_)
print(search.best_params_)
#pruint(pd.DataFrame.from_dict(search.cv_results_))
#predictions.append(search.best_estimator_.predict(test_x))

In [ ]:

##!!!!!!!! this is the pipeline!!!
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm, tree
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
best_estimators = []
from sklearn.model_selection import RandomizedSearchCV
#accuracies = []


results = []
predictions = []
X_shuffled, y_shuffled, groups_shuffled = shuffle(X,y,groups ,random_state=42)
cv = GroupKFold(n_splits=5)
pipe = Pipeline([
    ('scaler',StandardScaler()), 
    ('classifier', PipelineHelper([
        ('lr', LogisticRegression(class_weight='balanced')),
      #  ('forest', RandomForestClassifier(class_weight='balanced')),
        #('svm', SVC(class_weight='balanced'))

    ])),
    ])
params = {
    'classifier__selected_model':pipe.named_steps['classifier'].generate({
        'lr__penalty':['l1','l2'], 'lr__solver':['liblinear'],
                        'lr__C':np.logspace(-4,4,20), 
       # 'forest__n_estimators':[int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)],
       # 'forest__max_features':['auto', 'sqrt'], 'forest__max_depth':[int(x) for x in np.linspace(10, 110, num = 11)],
       # 'forest__min_samples_split':[2, 5, 10], 'forest__min_samples_leaf':[1, 2, 4], 'forest__bootstrap':[True, False],
        # 'svm__kernel': ['linear'],'svm__C':[0.001, 0.01, 0.1, 1, 10, 100], 'svm__gamma':[0.001, 0.01, 0.1, 1, 10, 100],
        
        
    })
}

search = RandomizedSearchCV(pipe, params, scoring = ['accuracy','balanced_accuracy','f1', 'precision','recall'],refit='balanced_accuracy', n_iter=30,
                            verbose = 10, cv =cv, n_jobs=-1)

search.fit(X_shuffled, y_shuffled, groups = groups_shuffled)
print(search.best_score_)
print(search.best_params_)
#pruint(pd.DataFrame.from_dict(search.cv_results_))
#predictions.append(search.best_estimator_.predict(test_x))

In [ ]:
df_grid_search = pd.DataFrame(search.cv_results_)
df_grid_search = df_grid_search.set_index('params')[['rank_test_accuracy', 'mean_test_accuracy', 'mean_test_f1','mean_test_precision', 'mean_test_recall']]
df_grid_search.sort_values(by = 'mean_test_f1', ascending=False).head(50)

In [ ]:
X_shuffled, y_shuffled, groups_shuffled = shuffle(X,y,groups ,random_state=42)

scoring = ['accuracy', 'recall','precision', 'f1']
scaler = preprocessing.StandardScaler()
clf = LogisticRegression(C= 0.004832930238571752,penalty='l2',solver='liblinear', class_weight='balanced')
pipe = Pipeline(steps=[("scaler", scaler), ("classifier", clf)])
cv = GroupKFold(n_splits=5)
scores = cross_validate(pipe, X_shuffled, y_shuffled,groups=groups_shuffled, cv=cv, scoring=scoring, return_estimator=True)

In [ ]:
data

In [ ]:
np.mean(scores['test_accuracy']), 

In [ ]:
np.mean(scores['test_precision']), np.mean(scores['test_accuracy']), np.mean(scores['test_f1']), np.mean(scores['test_recall']),

In [ ]:
f_importances = [x.named_steps['classifier'].coef_[0] for x in  scores['estimator']]
np.mean(f_importances,axis=0)

In [ ]:
features = pd.Series(np.mean(f_importances, axis=0), index=data.columns[:-1])
figure = features.abs().nlargest(20).plot(kind='barh')
#plt.savefig('child_meal_prosodic.png', bbox_inches='tight')

In [ ]:
from scipy import stats
clf = LogisticRegression(C = 0.012742749857031334, solver= 'liblinear',penalty='l1',class_weight='balanced')
def permutation_test(X,y,groups, clf):
    _scores = []
    cv = GroupKFold(n_splits=5)
    scaler = preprocessing.StandardScaler()
    pipe = Pipeline(steps=[("scaler", scaler), ("clf", clf)])
    scaler = preprocessing.StandardScaler()
    for i in range(0,100):
        y_shuffled,groups_shuffled= shuffle(y,groups,random_state=i)
        scoring = ['accuracy', 'recall','precision','f1']
        scores = cross_validate(pipe, X, y_shuffled,groups=groups_shuffled, cv=cv, scoring=scoring, n_jobs=-1)
        _scores.append(np.mean(scores['test_accuracy']))
    X_shuffled,y_shuffled,groups_shuffled= shuffle(X,y,groups,random_state=42)
    scoring = ['accuracy', 'recall','precision','f1']
    score_no_perm = cross_validate(pipe, X_shuffled, y_shuffled,groups=groups_shuffled,scoring=scoring, cv=cv)
    t_test = stats.ttest_1samp(_scores, np.mean(score_no_perm['test_accuracy']))
    return(_scores,score_no_perm['test_accuracy'], t_test.pvalue)
scores,no_perm, p_value = permutation_test(X,y, groups, clf)

In [ ]:
no_perm

In [ ]:
sme = SMOTEENN(random_state=42)

In [ ]:
kf = GroupKFold(n_splits=5)
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
accuracies = [] 
precisions = []
recalls = []
specifities = []
for fold, (train_index, test_index) in enumerate(kf.split(X,y,groups), 1):
    print(groups[test_index])
    X_train = X[train_index]
    y_train = y[train_index]  # Based on your code, you might need a ravel call here, but I would look into how you're generating your y
    X_test = X[test_index]
    y_test = y[test_index]  # See comment on ravel and  y_train
    sm = SMOTEENN(random_state=42)
    X_train_oversampled, y_train_oversampled = sm.fit_sample(X_train, y_train)
    pipe = Pipeline([('scaler', StandardScaler()), ('svc', SVC(kernel='linear',C=10, gamma=0.1, class_weight='balanced'))])
    pipe.fit(X_train_oversampled, y_train_oversampled)  
    y_pred = pipe.predict(X_test)
    print(f'For fold {fold}:')
    print(f'Accuracy: {pipe.score(X_test, y_test)}')
    accuracies.append(pipe.score(X_test, y_test))
    precisions.append(precision_score(y_test, pipe.predict(X_test)))
    recalls.append(recall_score(y_test, pipe.predict(X_test)))
    tn, fp, fn, tp = confusion_matrix(y_test, pipe.predict(X_test)).ravel()
    specifities.append(tn / (tn+fp))


In [ ]:
np.mean(accuracies), np.mean(precisions), np.mean(recalls), np.mean(specifities)

In [ ]:
steps = [('over', SMOTE()), ('model', LogisticRegression())]
pipeline = Pipeline(steps=steps)
# evaluate pipeline
cv = GroupKFold(n_splits=5)
scores = cross_val_score(pipeline, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
score =  mean(scores)

In [ ]:
from imblearn.pipeline import Pipeline as imbpipeline
from imblearn.under_sampling import EditedNearestNeighbours 
cv = GroupKFold(n_splits=5)
pipeline = imbpipeline(steps = [['smote',  EditedNearestNeighbours()],
                                ['scaler', StandardScaler()],
                                ['svm', SVC(class_weight='balanced')]])

param_grid = {'svm__C':[0.001, 0.01, 0.1, 1, 10, 100], 'svm__gamma':[0.001, 0.01, 0.1, 1, 10, 100],'svm__kernel':['linear']}
grid_search = GridSearchCV(estimator=pipeline,
                           param_grid=param_grid,
                           scoring=['accuracy','recall','f1'],
                           cv=cv,
                           n_jobs=-1, refit='f1')
grid_search.fit(X,y, groups=groups)

In [ ]:
df_grid_search = pd.DataFrame(grid_search.cv_results_)
df_grid_search = df_grid_search.set_index('params')[['rank_test_accuracy', 'mean_test_accuracy', 'mean_test_f1', 'mean_test_recall']]
df_grid_search.sort_values(by = 'rank_test_accuracy').head(50)

In [ ]:
grid_search.best_params_

In [ ]:
import autosklearn.classification